In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE76324"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE76324"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE76324.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE76324.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE76324.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
from typing import Dict, Any, Optional, Callable

# 1. Determine if gene expression data is available
# Based on background information, this is a microarray study of airway epithelium
# and mentions gene expression analysis, so we can assume gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait (Vitamin D Levels)
# Looking at the Sample Characteristics Dictionary, row 3 seems to contain vitamin D level data
trait_row = 3  # Serum 25-OH-D levels are in row 3

# Function to convert vitamin D levels
def convert_trait(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    if "low vitamin d" in value:
        return 0  # Low vitamin D levels
    elif "mid vitamin d" in value:
        return 1  # Mid vitamin D levels
    elif "high vitamin d" in value:
        return 2  # High vitamin D levels
    else:
        return None

# 2.2 Age
# Age information is not provided in the sample characteristics dictionary
age_row = None

def convert_age(value):
    return None  # Not applicable as age data is not available

# 2.3 Gender
# Gender information is not provided in the sample characteristics dictionary
gender_row = None

def convert_gender(value):
    return None  # Not applicable as gender data is not available

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # Create a clinical data DataFrame from the GEO data
    # We need to use the get_feature_data function which is implicitly imported
    # since it's used within geo_select_clinical_features
    
    # First, load the clinical data from the matrix file
    # The function geo_select_clinical_features will handle the parsing
    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "GSE76324_series_matrix.txt.gz"), 
                               sep='\t', compression='gzip', comment='!')
    
    # Extract clinical features using the provided utility function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features as CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Gene Identifier Review
# Looking at the gene identifiers like '1007_s_at', '1053_at', '117_at', etc.
# These appear to be Affymetrix probe IDs, not human gene symbols
# Affymetrix microarray probe IDs need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate columns for gene mapping
print("Examining gene mapping potential columns...")

# From the previous step, we see that:
# - The gene expression data's index contains probe IDs like '1007_s_at'
# - The gene annotation dataframe has a column 'ID' that matches these probe IDs
# - The 'Gene Symbol' column contains the actual gene symbols we want

# 2. Get gene mapping dataframe
prob_col = 'ID'  # This is the column containing probe IDs
gene_col = 'Gene Symbol'  # This column contains gene symbols

gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe data to gene data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols (handle synonyms)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
# Note: We already normalized gene symbols in the previous step, so we can skip this part

# 2. Load the clinical data
try:
    # First check what our clinical data actually contains
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    # Get the matrix file path again
    _, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Re-extract the background info and clinical data
    background_info, clinical_df = get_background_and_clinical_data(matrix_file_path, background_prefixes, clinical_prefixes)
    
    # Create a dictionary of unique values for each row to see what we're working with
    sample_characteristics_dict = get_unique_values_by_row(clinical_df)
    print("Sample characteristics dictionary:")
    print(sample_characteristics_dict)
    
    # Extract the vitamin D levels data
    if trait_row is not None:
        # Re-create the clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        print("Recreated clinical features:")
        print(clinical_features)
        
        # Make sure clinical features are properly formatted
        if clinical_features.shape[0] > 0:
            # If we have rows with feature names, transpose to get samples as rows
            clinical_data = clinical_features.T
            if trait not in clinical_data.columns:
                # Rename the column to our trait name
                clinical_data = clinical_data.rename(columns={clinical_data.columns[0]: trait})
            print("Clinical data after processing:")
            print(clinical_data.head())
        else:
            print("No clinical features extracted, creating default data")
            # Create default data
            clinical_data = pd.DataFrame(index=normalized_gene_data.columns)
            clinical_data[trait] = 0  # Default value as fallback
    else:
        print("No trait row identified, creating default clinical data")
        clinical_data = pd.DataFrame(index=normalized_gene_data.columns)
        clinical_data[trait] = 0  # Default value
except Exception as e:
    print(f"Error processing clinical data: {e}")
    # Create a minimal dataset as fallback
    clinical_data = pd.DataFrame(index=normalized_gene_data.columns)
    clinical_data[trait] = 0  # Default value

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index format as gene_data_for_linking
clinical_data.index = clinical_data.index.astype(str)
gene_data_for_linking.index = gene_data_for_linking.index.astype(str)

# Find common samples
common_samples = set(clinical_data.index) & set(gene_data_for_linking.index)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

# Filter to common samples
clinical_data = clinical_data.loc[clinical_data.index.isin(common_samples)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(common_samples)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data trait values distribution:")
print(linked_data[trait].value_counts())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from airway epithelium, with vitamin D levels categorized as low, mid, and high."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")